<a href="https://colab.research.google.com/github/njucs/med/blob/master/PM/PM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 授权 Colab 访问 Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/'Colab Notebooks'/Ophthalmology/PathologicMyopia/dataset
!pwd

/content/drive/MyDrive/Colab Notebooks/Ophthalmology/PathologicMyopia
/content/drive/MyDrive/Colab Notebooks/Ophthalmology/PathologicMyopia


In [ ]:
!pwd
!unzip PALM-Validation400.zip

In [ ]:
%cd PALM-Training400
!ls

/content/drive/MyDrive/Colab Notebooks/Ophthalmology/PathologicMyopia/dataset/PALM-Training400
'PALM-Training400-Annotation-D&F.zip'	   PALM-Training400.zip
 PALM-Training400-Annotation-Lession.zip


In [ ]:
!unzip 'PALM-Training400-Annotation-D&F.zip'

Archive:  PALM-Training400-Annotation-D&F.zip
  inflating: Fovea_Location.xlsx     
  inflating: __MACOSX/._Fovea_Location.xlsx  
   creating: Disc_Masks/
  inflating: Disc_Masks/N0109.bmp    
  inflating: Disc_Masks/N0121.bmp    
  inflating: Disc_Masks/N0135.bmp    
  inflating: Disc_Masks/P0027.bmp    
  inflating: Disc_Masks/P0033.bmp    
  inflating: Disc_Masks/P0190.bmp    
  inflating: Disc_Masks/P0184.bmp    
  inflating: Disc_Masks/P0153.bmp    
  inflating: Disc_Masks/P0147.bmp    
  inflating: Disc_Masks/N0096.bmp    
  inflating: Disc_Masks/H0004.bmp    
  inflating: Disc_Masks/H0010.bmp    
  inflating: Disc_Masks/N0082.bmp    
  inflating: Disc_Masks/N0069.bmp    
  inflating: Disc_Masks/N0055.bmp    
  inflating: Disc_Masks/N0041.bmp    
  inflating: Disc_Masks/N0040.bmp    
  inflating: Disc_Masks/N0054.bmp    
  inflating: Disc_Masks/N0068.bmp    
  inflating: Disc_Masks/H0011.bmp    
  inflating: Disc_Masks/N0083.bmp    
  inflating: Disc_Masks/N0097.bmp    
  inflati